In [60]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pyproj
import geopandas as gpd
from shapely.geometry import Point
from sklearn.linear_model import Lasso,LassoCV,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [61]:
concat_data = pd.read_csv("new_concat_data.csv")

concat_data = concat_data.drop([
    '國中(中)', '國中(下)', # (不影響)
    "總樓層數",
    '最近金融機構',# external的資料比較好
    '最近碼頭', '最近國小', '最近國中', '最近高中', '最近大學及研究所', '最近幼稚園、托兒所', '最近傳統市場', '最近一般公園', '最近里鄰公園', '最近服務性設施', '最近加油站', '最近百貨公司', '最近娛樂設施',
    # '最近火車站', '最近客運站', '最近捷運站', '最近飯店', '最近高鐵站', '最近航空站', '最近主要道路建設', '最近專科', '最近生活百貨量販', '最近觀光遊憩設施', '最近停車場地', '最近行政機關', '最近醫療保健', '最近電力資源', '最近產業用水及設施', '最近工廠', '最近商店', '最近超級市場', '最近大型購物中心', '國小(明星)', '國中(高)'
], axis=1)
concat_data = pd.get_dummies(concat_data, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])

(alpha = 0.00099, max_iter = 50000) 最佳: 13.174007708514127

In [63]:
concat = concat_data
train_data, test_data = train_test_split(concat, random_state=777, train_size=0.8)

Y_train = train_data[['單價']]
X_train = train_data.drop(['單價'], axis=1)
Y_test = test_data[['單價']]
X_test = test_data.drop(['單價'], axis=1)

In [13]:
reg = Lasso(alpha = 0.00099, max_iter = 50000)
# reg = Lasso(alpha = model.alpha_)
reg.fit(X_train.values, Y_train['單價'].values)

Lasso(alpha=0.00099, max_iter=50000)

In [64]:
ridge = Ridge()
parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_reg = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_reg.fit(X_train.values, Y_train['單價'].values)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=4.61024e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.87623e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.84934e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.08918e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarn

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             scoring='neg_mean_squared_error')

In [65]:
predictions = ridge_reg.predict(X_test.values)
np.mean(np.abs((Y_test['單價'].values - predictions) / Y_test['單價'].values)) * 100

10.294948000465688

## 正式

In [47]:
concat_data = pd.read_csv("new_concat_data.csv")
origin_concat_data = pd.read_csv("concat_data.csv")
origin_concat_data = origin_concat_data.drop(['geometry','縱坐標','橫坐標'], axis=1)
test_data = origin_concat_data[11751:]

In [48]:
print(concat_data.shape)
print(origin_concat_data.shape)
new_data = pd.concat([concat_data, test_data])

(10905, 60)
(17627, 57)


In [51]:
new_data

,土地面積,移轉層次,屋齡,建物面積,車位面積,車位個數,主建物面積,陽台面積,附屬建物面積,lng,...,備註_露台66.76㎡,備註_露台:12.9平方公尺,備註_露台:15.87平方公尺,備註_露台:7.92,備註_露樑0.89㎡,備註_露臺12.72平方公尺,備註_露臺18.48㎡,備註_露臺4.4,備註_露臺面積：7.6m2；花台面積：2.96m2,備註_騎樓:4.73平方公尺
0,0.100134,7,24.166667,0.314204,-0.819326,0.0,-0.316131,0.608577,-0.438452,121.502124,...,0,0,0,0,0,0,0,0,0,0
1,0.181921,10,6.166667,0.423366,0.161624,1.0,-0.098871,-0.360620,1.525881,120.365799,...,0,0,0,0,0,0,0,0,0,0
2,0.085594,9,8.833333,0.164249,0.524653,1.0,-0.071147,0.315088,0.231984,121.462402,...,0,0,0,0,0,0,0,0,0,0
3,0.260510,2,39.333333,-0.411438,-0.819326,0.0,-0.371295,-0.618277,-0.438452,121.508963,...,0,0,0,0,0,0,0,0,0,0
4,0.856422,2,46.000000,-0.868770,-0.819326,0.0,-0.245408,-1.642077,-0.438452,121.422831,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17622,0.560113,5,0.833333,-0.788335,-0.016028,1.0,-0.867769,-1.078987,-0.438452,120.187896,...,0,0,0,0,0,0,0,0,0,0
17623,-1.020142,5,3.416667,-0.910137,0.799628,1.0,-1.107095,-0.754784,-0.438452,120.693103,...,0,0,0,0,0,0,0,0,0,0
17624,-0.662691,10,12.583333,-0.459699,0.004054,1.0,-0.474267,-0.131972,-0.438452,121.511111,...,0,0,0,0,0,0,0,0,0,0
17625,0.513752,2,36.250000,0.357294,-0.819326,0.0,0.926327,1.243333,-0.316749,121.583046,...,0,0,0,0,0,0,0,0,0,0


In [50]:
new_data = new_data.drop([
    '國中(中)', '國中(下)', # (不影響)
    "總樓層數", "最近金融機構距離", "最近廟宇距離",
    '最近金融機構',# external的資料比較好
    '最近碼頭', '最近國小', '最近國中', '最近高中', '最近大學及研究所', '最近幼稚園、托兒所', '最近傳統市場', '最近一般公園', '最近里鄰公園', '最近服務性設施', '最近加油站', '最近百貨公司', '最近娛樂設施',
    # '最近火車站', '最近客運站', '最近捷運站', '最近飯店', '最近高鐵站', '最近航空站', '最近主要道路建設', '最近專科', '最近生活百貨量販', '最近觀光遊憩設施', '最近停車場地', '最近行政機關', '最近醫療保健', '最近電力資源', '最近產業用水及設施', '最近工廠', '最近商店', '最近超級市場', '最近大型購物中心', '國小(明星)', '國中(高)'
], axis=1)
new_data = pd.get_dummies(new_data, columns=['縣市','鄉鎮市區','路名','主要用途','主要建材','建物型態','使用分區','備註'])

In [52]:
train_data = new_data[:10905]
test_data = new_data[10905:]

Y_train = train_data[['單價']]
X_train = train_data.drop(['單價'], axis=1)
X_test = test_data.drop(['單價'], axis=1)

In [56]:
ridge = Ridge()
parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,100]}
ridge_reg = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_reg.fit(X_train.values, Y_train['單價'].values)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.63815e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.38798e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=2.42025e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=3.66668e-25): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarn

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.001, 0.01, 1, 5, 10,
                                   20, 30, 35, 40, 45, 50, 55, 100]},
             scoring='neg_mean_squared_error')

In [58]:
predictions = ridge_reg.predict(X_test.values)
predictions

array([1.41719236, 1.55595112, 2.71826347, ..., 2.03950295, 3.22141944,
       2.39268142])

In [59]:
result = pd.read_csv("public_submission_template.csv")
result['predicted_price'] = predictions
result.to_csv("./result.csv", index=False)